In [2]:
%pip install -e /Users/yousuf/Documents/ucl-msc/meta-misinformation-detection

distutils: /usr/local/lib/python3.9/site-packages
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages
distutils: /usr/local/lib/python3.9/site-packages
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages
distutils: /usr/local/include/python3.9/UNKNOWN
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/include/python3.9
distutils: /usr/local/bin
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/bin
distutils: /usr/local
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9
user = False
home = None
root = None
prefix = None
Obtaining file:///Users/yousuf/Documents/ucl-msc/meta-misinformation-detection
  Attempting uninstall: metaphor
    Found existing installation: metaphor 0.0.1
    Uninstalling metaphor-0.0.1:
      Successfully uninstalled metaphor-0.0.1
  distutils: /usr/local/lib/python3.9/site-pac

In [1]:
from metaphor.models.common import Tokenizer, WordEmbedding, Glove, MeanPooler
sentences = ["Hello", "Hello World"]
t = Tokenizer()
t_sentences = t(sentences)
embedder = WordEmbedding(Glove(t).weights, MeanPooler(t))
pooled_embedding = embedder(t_sentences)

/usr/local/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


KeyboardInterrupt: 

In [1]:
import numpy as np 
np.random.seed(0)
np.random.randint(4)

0

In [5]:
np.random.seed(0)
x = np.arange(10)
np.random.permutation(x)

array([2, 8, 4, 9, 1, 6, 7, 3, 0, 5])

In [5]:
import re
text = "#Germanwings co-pilot suffered serious depression 6 years ago: German newspaper http://t.co/LXDzBdNdMB http://t.co/yTUNdHyEJY"
regex = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
text = re.sub(regex, "", text)
text.strip()

'#Germanwings co-pilot suffered serious depression 6 years ago: German newspaper'

In [3]:
import pandas as pd
df = pd.read_csv("/Users/yousuf/Documents/ucl-msc/meta-misinformation-detection/data/pheme/processed-pheme.csv")

In [7]:
df = df.dropna()
topics = sorted(list(set(df["topic"])))
len(topics)

9

In [12]:
topics = ['a', 'b', 'c']
data = {'topic': ['b','c','a','a','b']}
topic_indxs = [[i for i in range(len(data['topic'])) if topics.index(data['topic'][i]) == j] for j in range(len(topics))]

In [13]:
topic_indxs

[[2, 3], [0, 4], [1]]

In [16]:
topic_indxs = [
        [
            i
            for i in range(len(data["topic"]))
            if topics.index(data["topic"][i]) == j
        ]
        for j in range(len(topics))
    ]

In [17]:
topic_indxs

[[2, 3], [0, 4], [1]]

In [29]:
def filter_indices(t):
    return [[y for y in x if y in t] for x in topic_indxs]

ti = [0,1,2]
indices = [
    filter_indices(t)
    for t in [ti]
]
z = [
    [
        t for t in i
    ]
    for i in indices
]

In [32]:
z[0]

[[2], [0], [1]]

In [37]:
import numpy as np
x = np.array([1,2,3])
x[np.array([])]

IndexError: arrays used as indices must be of integer (or boolean) type